# Theoretical Overview


$$ P(class=i) = \frac{e^i}{\sum e^i} $$

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

# For reproducibility
torch.manual_seed(1)

In [6]:
# 바로 의사결정
z = torch.FloatTensor([1, 2, 3])
print(max(z))

# 확률값으로 의사결정
hypothesis = F.softmax(z, dim=0)
print(hypothesis)
# 확률값의 합은 1
hypothesis.sum()

tensor(3.)
tensor([0.0900, 0.2447, 0.6652])


tensor(1.)

# 1. Cross Entropy Loss (Low-level)

두 개의 확률분포가 주어져있을 때 두 확률분포가 얼마나 비슷한지 나타냄

$$ L = \frac{1}{N} \sum - y \log(\hat{y}) $$

$$ y = P(x) $$
$$\hat{y} = Q(x) $$ 

In [7]:
z = torch.rand(3, 5, requires_grad=True) # ( 3, 5 )
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

y = torch.randint(5, (3,)).long()
print(y)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)
tensor([0, 2, 1])


In [8]:
# 위의 결과값을 (index로 표현되어있음) one-hot vector로 표현하자

# 같은 size의 0으로 구성된 것 vector 생성
y_one_hot = torch.zeros_like(hypothesis)
# dimension 1에 대해서 y를 unsqueeze해서 vector크기를 맞춰주자
# unsqueeze : (3, ) => (3, 1)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [9]:
# y_one_hot : (3, 5)
# torch.log(hypothesis) : (3, 5)
# sum(dim=1) : (3, 1)
# mean() : (1, 1)

cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


In [13]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [15]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True) # 4차원 벡터 n개 , 예측할 값 3개
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 10000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 1000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/10000 Cost: 1.098612
Epoch 1000/10000 Cost: 0.738749
Epoch 2000/10000 Cost: 0.710759
Epoch 3000/10000 Cost: 0.697990
Epoch 4000/10000 Cost: 0.690640
Epoch 5000/10000 Cost: 0.685888
Epoch 6000/10000 Cost: 0.682576
Epoch 7000/10000 Cost: 0.680144
Epoch 8000/10000 Cost: 0.678285
Epoch 9000/10000 Cost: 0.676820
Epoch 10000/10000 Cost: 0.675637


# 2. Cross-entropy Loss with torch.nn.functional

In [12]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

# High level
F.nll_loss(F.log_softmax(z, dim=1), y)

# 이것도 한 번에 할 수 있다
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward>)

In [16]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3!

    def forward(self, x):
        return self.linear(x)
    
model = SoftmaxClassifierModel()

In [17]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609258
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358733
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269521
Epoch 1000/1000 Cost: 0.241922
